In [1]:
from pyomo.environ import (
    ConcreteModel,
    Objective,
    Expression,
    value,
    Var,
    Param,
    Constraint,
    Set,
    Var,
    Block,
    SolverFactory,
    TransformationFactory,
    assert_optimal_termination,
    check_optimal_termination,
    log,
    log10,
    units as pyunits,
)

from pyomo.network import Port
from idaes.core import FlowsheetBlock
from idaes.core.solvers.get_solver import get_solver

from idaes.core.util.model_statistics import *
from idaes.core.util.scaling import *

from idaes.core import UnitModelCostingBlock

from watertap.core.zero_order_costing import ZeroOrderCosting, _get_tech_parameters
from watertap.core.util.infeasible import *
from watertap.costing import WaterTAPCosting

import json
from os.path import join, dirname
from math import floor, ceil

import pytest

from io import StringIO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from IPython.display import clear_output
from copy import deepcopy
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.property_models.water_prop_pack import WaterParameterBlock
from watertap_contrib.seto.costing import SETOZeroOrderCosting, SETOWaterTAPCosting

# from watertap_contrib.seto.solar_models.zero_order import PhotovoltaicZO
# from watertap_contrib.seto.energy import solar_energy
# from watertap_contrib.seto.core import SETODatabase, PySAMWaterTAP
from watertap_contrib.seto.unit_models import ChemicalSoftening0D
from watertap_contrib.seto.property_models.chemical_softening_prop_pack import (
    ChemSofteningParameterBlock,
    ChemSofteningStateBlock
)

# from watertap_contrib.seto.costing.solar import photovoltaic

solver = get_solver()

def get_ion_config(ions):
    #neutral_solutes = ["TSS", "TDS", "TOC", "NH3", "SiO2"]
    if not isinstance(ions, list):
        ions = [ions]
    #ions = ions + neutral_solutes

    mw_data = {
        "Na_+": 23e-3,
        "Ca_2+": 40e-3,
        "Cl_-": 35e-3,
        "Mg_2+": 24e-3,
        "SO4_2-": 96e-3,
        "NH3": 17.03e-3,
        "SiO2": 60.08e-3,
        "HCO3_-": 61.02e-3,
        "Alkalinity_2-": 31.01736e-3,
        "CO3_2-": 60.01e-3,
        "CO2": 44e-3,
    }
    charge_data = {
        "Na_+": 1,
        "Ca_2+": 2,
        "Cl_-": -1,
        "Mg_2+": 2,
        "SO4_2-": -2,
        "HCO3_-": -1,
        "CO3_2-": -2,
        "Alkalinity_2-": -2,
    }
    ion_config = {
        "solute_list": [],
        "mw_data": {"H2O": 18e-3},
        "charge": {},
    }

    for ion in ions:
        ion_config["solute_list"].append(ion)

        if ion in charge_data.keys():
            ion_config["charge"][ion] = charge_data[ion]
        if ion in mw_data.keys():
            ion_config["mw_data"][ion] = mw_data[ion]
    return ion_config


In [2]:
component_list = ["Ca_2+","Mg_2+","SiO2"]
input_config_dict = get_ion_config(component_list)
input_config_dict

{'solute_list': ['Ca_2+', 'Mg_2+', 'SiO2'],
 'mw_data': {'H2O': 0.018, 'Ca_2+': 0.04, 'Mg_2+': 0.024, 'SiO2': 0.06008},
 'charge': {'Ca_2+': 2, 'Mg_2+': 2}}

In [3]:
ca_in = 0.075 * pyunits.kg / pyunits.m**3  # g/L = kg/m3
mg_in = 0.0061 * pyunits.kg / pyunits.m**3  # g/L = kg/m3
sio2_in = 0.054 * pyunits.kg / pyunits.m**3  # g/L = kg/m3

q_in = 50000 * pyunits.m**3 / pyunits.day  # m3/d
rho = 1000 * pyunits.kg / pyunits.m**3

pH_in = 12
temp_in = 293.15  # K

ca_mass_flow = pyunits.convert((ca_in * q_in), to_units=pyunits.kg / pyunits.s)()
mg_mass_flow = pyunits.convert((mg_in * q_in), to_units=pyunits.kg / pyunits.s)()
sio2_mass_flow = pyunits.convert((sio2_in * q_in), to_units=pyunits.kg / pyunits.s)()
h2o_mass_flow = pyunits.convert((rho * q_in), to_units=pyunits.kg / pyunits.s)()



In [4]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = ChemSofteningParameterBlock(**input_config_dict)
m.fs.soft = soft = ChemicalSoftening0D(
    property_package=m.fs.properties, silica_removal= True,softening_procedure_type= 'single_stage_lime'
)

prop_in = m.fs.soft.properties_in[0]
prop_out = m.fs.soft.properties_out[0]
prop_waste = m.fs.soft.properties_waste[0]

print(f"DOF = {degrees_of_freedom(m)}")


Unfixed variables 54
44
DOF = 10


In [5]:
print(activated_equalities_set(m))

ComponentSet(['fs.soft.eq_recovery (id=2763222486928)', 'fs.soft.eq_waste_flow (id=2763222488384)', 'fs.soft.eq_mass_balance[H2O] (id=2763222287264)', 'fs.soft.eq_mass_balance[Ca_2+] (id=2763222287360)', 'fs.soft.eq_mass_balance[Mg_2+] (id=2763222287456)', 'fs.soft.eq_mass_balance[SiO2] (id=2763222287552)', 'fs.soft.eq_effluent_ca (id=2763222488496)', 'fs.soft.eq_effluent_total_hardness (id=2763222488608)', 'fs.soft.eq_effluent_alkalinity (id=2763222489840)', 'fs.soft.eq_mg_add (id=2763222491184)', 'fs.soft.eq_effluent_mg[Mg_2+] (id=2763222289568)', 'fs.soft.eq_volume_mixer (id=2763222491408)', 'fs.soft.eq_volume_floc (id=2763222491520)', 'fs.soft.eq_volume_sed (id=2763222491632)', 'fs.soft.eq_volume_recarb (id=2763222491744)', 'fs.soft.eq_excess_lime (id=2763222491856)', 'fs.soft.properties_in[0.0].eq_flow_vol_phase[Liq] (id=2763221925696)', 'fs.soft.properties_in[0.0].eq_dens_mass_phase[Liq] (id=2763221925792)', 'fs.soft.properties_in[0.0].eq_conc_mass_caco3_comp[Ca_2+] (id=276322228

In [5]:
print((variables_in_activated_equalities_set(m)))

ComponentSet(['fs.soft.properties_in[0.0].flow_vol_phase[Liq] (id=1715770150912)', 'fs.soft.properties_out[0.0].flow_vol_phase[Liq] (id=1715770151360)', 'fs.soft.properties_waste[0.0].flow_vol_phase[Liq] (id=1715770151808)', 'fs.soft.properties_in[0.0].flow_mass_phase_comp[Liq,H2O] (id=1715769784496)', 'fs.soft.properties_out[0.0].flow_mass_phase_comp[Liq,H2O] (id=1715769785840)', 'fs.soft.properties_waste[0.0].flow_mass_phase_comp[Liq,H2O] (id=1715770147776)', 'fs.soft.properties_in[0.0].flow_mass_phase_comp[Liq,Ca_2+] (id=1715769784384)', 'fs.soft.properties_out[0.0].flow_mass_phase_comp[Liq,Ca_2+] (id=1715769785168)', 'fs.soft.properties_waste[0.0].flow_mass_phase_comp[Liq,Ca_2+] (id=1715770147104)', 'fs.soft.properties_in[0.0].flow_mass_phase_comp[Liq,Mg_2+] (id=1715769784608)', 'fs.soft.properties_out[0.0].flow_mass_phase_comp[Liq,Mg_2+] (id=1715769785952)', 'fs.soft.properties_waste[0.0].flow_mass_phase_comp[Liq,Mg_2+] (id=1715770147888)', 'fs.soft.properties_in[0.0].flow_mass_ph

In [11]:
prop_in.flow_mass_phase_comp["Liq", "Ca_2+"].fix(ca_mass_flow)
set_scaling_factor(prop_in.flow_mass_phase_comp["Liq", "Ca_2+"], 100)
prop_in.flow_mass_phase_comp["Liq", "Mg_2+"].fix(mg_mass_flow)
set_scaling_factor(prop_in.flow_mass_phase_comp["Liq", "Mg_2+"], 100)
prop_in.flow_mass_phase_comp["Liq", "SiO2"].fix(sio2_mass_flow)
set_scaling_factor(prop_in.flow_mass_phase_comp["Liq", "SiO2"], 100)
prop_in.flow_mass_phase_comp["Liq", "H2O"].fix(h2o_mass_flow)
set_scaling_factor(prop_in.flow_mass_phase_comp["Liq", "H2O"], 1e-2)

prop_in.alkalinity.fix(0.195)

soft.retention_time_mixer.fix()
soft.retention_time_floc.fix()
soft.retention_time_sed.fix()
soft.retention_time_recarb.fix()


print(f"DOF = {degrees_of_freedom(prop_in)}")




Unfixed variables 12
12
DOF = 0


In [12]:
print((unfixed_variables_in_activated_equalities_set(m)))

ComponentSet(['fs.soft.properties_in[0.0].flow_vol_phase[Liq] (id=2456149144656)', 'fs.soft.properties_out[0.0].flow_vol_phase[Liq] (id=2456149145104)', 'fs.soft.properties_waste[0.0].flow_vol_phase[Liq] (id=2456149145552)', 'fs.soft.properties_out[0.0].flow_mass_phase_comp[Liq,H2O] (id=2456149140176)', 'fs.soft.properties_waste[0.0].flow_mass_phase_comp[Liq,H2O] (id=2456149141520)', 'fs.soft.properties_out[0.0].flow_mass_phase_comp[Liq,Ca_2+] (id=2456149139504)', 'fs.soft.properties_waste[0.0].flow_mass_phase_comp[Liq,Ca_2+] (id=2456149140848)', 'fs.soft.properties_out[0.0].flow_mass_phase_comp[Liq,Mg_2+] (id=2456149140288)', 'fs.soft.properties_waste[0.0].flow_mass_phase_comp[Liq,Mg_2+] (id=2456149141632)', 'fs.soft.properties_out[0.0].flow_mass_phase_comp[Liq,SiO2] (id=2456149140400)', 'fs.soft.properties_waste[0.0].flow_mass_phase_comp[Liq,SiO2] (id=2456149141744)', 'fs.soft.properties_out[0.0].conc_mass_caco3_comp[Ca_2+] (id=2456149146224)', 'fs.soft.properties_out[0.0].conc_mass_

In [9]:
prop_in.pH.fix(pH_in)
prop_in.pressure.fix(101325)

m.fs.soft.initialize()
print(f"DOF = {degrees_of_freedom(m)}")

Unfixed variables 12
12
2023-02-14 15:08:22 [INFO] idaes.init.fs.soft: Initialization Step 1a Complete.
Unfixed variables 15
14
Unfixed variables 41
44
DOF = -3


In [8]:
prop_in.temperature.fix(temp_in)

# prop_in.alkalinity.fix(0.195)
# m.fs.soft.excess_lime.fix(170)


In [9]:
results = solver.solve(m)

print(f"DOF = {degrees_of_freedom(m)}")
print(results.solver.termination_condition.swapcase())
print_infeasible_constraints(m)


    model.name="unknown";
      - termination condition: other
      - message from solver: Too few degrees of freedom (rethrown)!
Unfixed variables 41
44
DOF = -3
OTHER
CONSTR fs.soft.eq_recovery: -0.0010000000000000009 =/= 0.0
CONSTR fs.soft.eq_waste_flow: 0.005787818865740757 =/= 0.0
CONSTR fs.soft.eq_mass_balance[H2O]: -0.10000000000002274 =/= 0.0
CONSTR fs.soft.eq_mass_balance[Ca_2+]: -0.09999999999999999 =/= 0.0
CONSTR fs.soft.eq_mass_balance[Mg_2+]: -0.1 =/= 0.0
CONSTR fs.soft.eq_mass_balance[SiO2]: -0.1 =/= 0.0
CONSTR fs.soft.eq_effluent_ca: 100 =/= 0.03
CONSTR fs.soft.eq_effluent_total_hardness: 99.97458676666116 =/= 0.0
CONSTR fs.soft.eq_effluent_alkalinity: -99.81252532782821 =/= 0.0
CONSTR fs.soft.eq_mg_add: 9.508666379172174 =/= 0.0
CONSTR fs.soft.eq_effluent_mg[Mg_2+]: 89.97458676666116 =/= 0.0
CONSTR fs.soft.eq_volume_mixer: 9.421218113425926 =/= 0.0
CONSTR fs.soft.eq_volume_floc: 3.054617361111111 =/= 0.0
CONSTR fs.soft.eq_volume_sed: -76.81728298611111 =/= 0.0
CONSTR f

In [10]:
prop_waste.display()

Block fs.soft.properties_waste[0.0]

  Variables:
    flow_mass_phase_comp : Mole flow rate
        Size=4, Index=fs.soft.properties_waste[0.0].flow_mass_phase_comp_index, Units=kg/s
        Key              : Lower : Value : Upper : Fixed : Stale : Domain
        ('Liq', 'Ca_2+') :     0 :   0.1 :  None : False :  True : NonNegativeReals
          ('Liq', 'H2O') :     0 :   0.1 :  None : False :  True : NonNegativeReals
        ('Liq', 'Mg_2+') :     0 :   0.1 :  None : False :  True : NonNegativeReals
         ('Liq', 'SiO2') :     0 :   0.1 :  None : False :  True : NonNegativeReals
    temperature : State temperature
        Size=1, Index=None, Units=K
        Key  : Lower  : Value  : Upper  : Fixed : Stale : Domain
        None : 273.15 : 298.15 : 373.15 : False :  True : NonNegativeReals
    pressure : State pressure
        Size=1, Index=None, Units=Pa
        Key  : Lower    : Value  : Upper : Fixed : Stale : Domain
        None : 100000.0 : 101325 :  None : False :  True : Non